# ADVERSARIAL EXAMPLES IMPROVE IMAGE RECOGNITION

### Importing Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### Installing the Foolbox library
#### It will be used to generate adversarial attacks and to attack the networks

In [ ]:
!pip install foolbox==3.1.1

### Importing the needed libraries and modules

In [ ]:
import os, shutil
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import numpy as np
import foolbox as fb

from math import ceil
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.utils.tensorboard import SummaryWriter

### Setting a manual seed for reproducibility

In [ ]:
torch.manual_seed(1234)

### Defining the needed data transformations and creating the DataLoader

In [ ]:
def get_data(dataset, batch_size, test_batch_size=256): 
  transform = list()

  if dataset == 'MNIST':
    transform.append(torchvision.transforms.ToTensor()) 
    transform.append(torchvision.transforms.Lambda(lambda x: x.repeat(3,1,1)))
    transform.append(torchvision.transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]))
    transform = torchvision.transforms.Compose(transform)
    full_training_data = torchvision.datasets.MNIST('./data', train=True, transform=transform, download=True) 
    test_data = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True) 
  
  elif dataset == 'SVHN':
    transform.append(torchvision.transforms.ToTensor()) 
    transform.append(torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]))
    transform = torchvision.transforms.Compose(transform)
    full_training_data = torchvision.datasets.SVHN('./data', split='train', transform=transform, download=True) 
    test_data = torchvision.datasets.SVHN('./data', split='test', transform=transform, download=True) 

  elif dataset == 'CIFAR':
    transform.append(torchvision.transforms.ToTensor()) 
    #transform.append(torchvision.transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],std=[0.247, 0.243, 0.261]))
    transform.append(torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]))
    transform = torchvision.transforms.Compose(transform)
    full_training_data = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True) 
    test_data = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True) 
  
  # Create train and validation splits
  num_samples = len(full_training_data)
  training_samples = int(num_samples*0.5+1)
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)
  val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False)
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False) 
  
  return train_loader, val_loader, test_loader


### Defining the scaling parameters for EfficientNet architecture

In [ ]:
# SCALING PARAMETERS FOR EFFICIENTNET

base_model = [
    # expand_ratio, channels, repeats, stride, kernel_size
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

phi_values = {
    # tuple of: (phi_value, resolution, drop_rate)
    "b0": (0, 224, 0.2),  # alpha, beta, gamma, depth = alpha ** phi
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

### Defining the CNN, Squeeze-excitation and inverted-residual blocks for the EfficientNet architecture

In [ ]:
# DEFINITION OF CNN, SQUEEZE-EXCITATION AND INVERTED-RESIDUAL BLOCKS

class CNNBlock(nn.Module):
    def __init__(
            self, in_channels, out_channels, kernel_size, stride, padding, mode, groups=1,
    ):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            groups=groups,
            bias = False,
        )
        self.mode = mode
        self.bn = nn.BatchNorm2d(out_channels)
        if self.mode == 'advprop':
          self.bn_adv = nn.BatchNorm2d(out_channels)
        if self.mode == 'advprop2':
          self.bn_adv = nn.BatchNorm2d(out_channels)
          self.bn_adv2 = nn.BatchNorm2d(out_channels)

        self.silu = nn.SiLU() 

    def forward(self, x):
      if self.training:
        x = self.cnn(x)

        if self.mode == 'vanilla':
          x = self.silu(self.bn(x))
    
        elif self.mode == 'advprop':
          x_clean, x_adv = torch.split(x, split_size_or_sections = x.shape[0] // 2, dim = 0)
          x_clean = self.bn(x_clean)
          x_adv = self.bn_adv(x_adv)
          x = torch.cat((x_clean, x_adv), dim = 0)
          x = self.silu(x)
        
        elif self.mode == 'advprop2':
          x_clean, x_adv, x_adv2 = torch.split(x,split_size_or_sections=x.shape[0] // 3, dim = 0)
          x_clean = self.bn(x_clean)
          x_adv = self.bn_adv(x_adv)
          x_adv2 = self.bn_adv2(x_adv2)
          x = torch.cat((x_clean, x_adv, x_adv2), dim = 0)
  
      else:
        x=self.silu(self.bn(self.cnn(x)))
      
      return x

class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # C x H x W -> C x 1 x 1
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class InvertedResidualBlock(nn.Module):
    def __init__(
            self,
            mode,
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            expand_ratio,
            reduction = 4, 
            survival_prob = 0.8
            ):
      
        super(InvertedResidualBlock, self).__init__()

        self.mode = mode
        if self.mode == 'vanilla':
          self.bn = nn.BatchNorm2d(out_channels)
        elif self.mode == 'advprop':
          self.bn = nn.BatchNorm2d(out_channels)
          self.bn_adv = nn.BatchNorm2d(out_channels)
        elif self.mode == 'advprop2':
          self.bn = nn.BatchNorm2d(out_channels)
          self.bn_adv = nn.BatchNorm2d(out_channels)
          self.bn_adv2 = nn.BatchNorm2d(out_channels)

        self.survival_prob = 0.8
        self.use_residual = in_channels == out_channels and stride == 1
        hidden_dim = in_channels * expand_ratio
        self.expand = in_channels != hidden_dim
        reduced_dim = int(in_channels / reduction)

        if self.expand:
            self.expand_conv = CNNBlock(in_channels, 
                                        hidden_dim, 
                                        kernel_size = 3, 
                                        stride = 1, 
                                        padding = 1,
                                        mode = self.mode
                                        )

        self.conv = nn.Sequential(CNNBlock(hidden_dim, 
                                           hidden_dim, 
                                           kernel_size, 
                                           stride, 
                                           padding, 
                                           mode = self.mode,
                                           groups = hidden_dim),
                                  SqueezeExcitation(hidden_dim, reduced_dim),
                                  nn.Conv2d(hidden_dim, out_channels, 1, bias = False)
                                  )

    def stochastic_depth(self, x):
        if not self.training:
            return x

        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device = x.device) < self.survival_prob
        
        return torch.div(x, self.survival_prob) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs
        x = self.conv(x)

        if self.training:
          if self.mode == 'vanilla':
            x = self.bn(x)
          elif self.mode == 'advprop':
            x_clean, x_adv = torch.split(x, split_size_or_sections = x.shape[0] // 2, dim = 0) 
            x_clean = self.bn(x_clean)
            x_adv = self.bn_adv(x_adv)
            x = torch.cat((x_clean, x_adv), dim = 0)
          elif self.mode == 'advprop2':
            x_clean, x_adv, x_adv2 = torch.split(x, split_size_or_sections = x.shape[0] // 3, dim = 0)
            x_clean = self.bn(x_clean)
            x_adv = self.bn_adv(x_adv)
            x_adv2 = self.bn_adv2(x_adv2)
            x = torch.cat((x_clean, x_adv, x_adv2), dim = 0)
           
        else:
          x = self.bn(x)

        if self.use_residual:
            return self.stochastic_depth(x) + inputs
        else:
            return x #self.conv(x)


### Defining the EfficientNet class for building the whole architecture

In [ ]:
# DEFINITION OF THE ARCHITECTURE

class EfficientNet(nn.Module):
    def __init__(self, version, num_classes, mode):   
        super(EfficientNet, self).__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(version)
        last_channels = ceil(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.mode = mode
        self.features = self.create_features(width_factor, depth_factor, last_channels)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, num_classes),
        )

    def calculate_factors(self, version, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[version]
        depth_factor = alpha ** phi
        width_factor = beta ** phi
        return width_factor, depth_factor, drop_rate

    def create_features(self, width_factor, depth_factor, last_channels):
        channels = int(32 * width_factor)
        features = [CNNBlock(3, channels, 3, stride=2, padding=1, mode = self.mode)]
        in_channels = channels

        for expand_ratio, channels, repeats, stride, kernel_size in base_model:
            out_channels = 4*ceil(int(channels*width_factor) / 4)
            layers_repeats = ceil(repeats * depth_factor)

            for layer in range(layers_repeats):
                features.append(
                    InvertedResidualBlock(
                        self.mode,
                        in_channels,
                        out_channels,
                        expand_ratio=expand_ratio,
                        stride = stride if layer == 0 else 1,
                        kernel_size=kernel_size,
                        padding=kernel_size//2, 
                    )
                )
                in_channels = out_channels

        features.append(
            CNNBlock(in_channels, last_channels, kernel_size=1, stride=1, padding=0, mode = self.mode)
        )

        return nn.Sequential(*features)

    def forward(self, x):
        x = self.pool(self.features(x))
        return self.classifier(x.view(x.shape[0], -1))


### Defining the class for early stopping

In [ ]:
class EarlyStopping():
    def __init__(self, min_delta = 0, patience = 0):
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf
        self.stop_training = False
    def on_epoch_end(self, epoch, current_value):
        if np.greater(self.best, (current_value - self.min_delta)):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

### Definining the cross entropy cost function

In [ ]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

### Defining the stochastic gradient descent optimizer

In [ ]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

# def get_optimizer(net, lr, wd, momentum):
#   optimizer = torch.optim.RMSprop(net.parameters(), lr = lr, weight_decay = wd, momentum = momentum)
#   return optimizer

### Defining the learning rate scheduler

In [ ]:
# def get_scheduler(optimizer, step_size, gamma):
#   scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)
#   return scheduler

### Defining the train and test functions

In [ ]:
def test(net, data_loader, cost_function, device='cuda'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() 
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() 
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100




def train(net, mode, data_loader, optimizer, cost_function, device='cuda'):   #scheduler
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  
  net.train() 
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)

    if mode == 'advprop':
      attack = fb.attacks.PGD(rel_stepsize = 1, steps = 1)
      raw, clipped, is_adv = attack(foolbox_model_vanilla, inputs, targets, epsilons=0.003)
      raw = raw.to(device)
      inputs = torch.cat((inputs, raw), dim = 0) 

    if mode == 'advprop2':
      attack = fb.attacks.PGD(rel_stepsize = 1, steps = 1)
      attack2 = fb.attacks.VirtualAdversarialAttack(steps = 1, xi = 0.13)
      raw, clipped, is_adv = attack(foolbox_model_vanilla, inputs, targets, epsilons=0.0157)
      raw2, clipped2, is_adv2 = attack2(foolbox_model_vanilla, inputs, targets, epsilons=0.26)
      raw = raw.to(device)
      raw2 = raw2.to(device)
      inputs=torch.cat((inputs, raw, raw2),dim=0)
      
    # Forward pass
    outputs = net(inputs)

    # Apply the loss
    if mode == 'vanilla':
      loss = cost_function(outputs, targets)
    elif mode == 'advprop':
      out1, out2 = torch.split(outputs, split_size_or_sections = outputs.shape[0] // 2, dim = 0)
      loss1 = cost_function(out1, targets)
      loss2 = cost_function(out2, targets)
      loss = loss1 + loss2
      targets = torch.cat((targets, targets), dim = 0)
    elif mode == 'advprop2':
      out1, out2, out3 = torch.split(outputs, split_size_or_sections = outputs.shape[0] // 3, dim = 0)
      loss1 = cost_function(out1, targets)
      loss2 = cost_function(out2, targets)
      loss3 = cost_function(out3, targets)
      loss = loss1 + loss2 + loss3
      targets = torch.cat((targets, targets, targets),dim = 0) 

    loss.backward()
    
    optimizer.step()

    # Decaying the learning rate
    # scheduler.step()
    
    optimizer.zero_grad()

    samples += inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(dim=1) 

    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

### Wrapping all up

In [ ]:
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

def main(dataset,
         mode,
         batch_size = 128, 
         #input_dim=28*28, 
         #hidden_dim=100, 
         #output_dim=10, 
         device = 'cuda:0', 
         learning_rate = 0.01,  #0.256 
         weight_decay = 0.000001,  #0.9
         momentum = 0.9, 
         epochs = 50
         ):
  
  from torch.utils.tensorboard import SummaryWriter
  # Creates a logger for the experiment
  writer = SummaryWriter(log_dir = "runs/exp1")

  # Gets DataLoaders
  train_loader, val_loader, test_loader = get_data(dataset, batch_size)
  
  # Defining needed parameters
  version = 'b0'
  num_classes = 10

  net = EfficientNet(version = version, 
                     num_classes = num_classes,
                     mode = mode).to(device)   
  
  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

  # Instantiates the learning rate scheduler
  # scheduler = get_scheduler(optimizer, step_size = 2.4, gamma=0.001)
  
  # Creates the cost function
  cost_function = get_cost_function()

  # Computes evaluation results before training
  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  # Logs to TensorBoard
  log_values(writer, -1, train_loss, train_accuracy, "Train")
  log_values(writer, -1, val_loss, val_accuracy, "Validation")
  log_values(writer, -1, test_loss, test_accuracy, "Test")

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  all_early_stopping = EarlyStopping(patience = 4)
  all_early_stopping.stop_training = False
  # For each epoch, train the network and then compute evaluation results
  for e in range(epochs):
    if all_early_stopping.stop_training: 
      break
      print("Reached Early Stopping Patience at Epoch {}".format(e))

    train_loss, train_accuracy = train(net, mode, train_loader, optimizer, cost_function)   #scheduler
    val_loss, val_accuracy = test(net, val_loader, cost_function)

    all_early_stopping.on_epoch_end(epoch = (e + 1), current_value = round(val_loss,5))
    if all_early_stopping.wait == 0: 
      bestModel = net
      # current_time = datetime.now().strftime('%b%d_%H-%M-%S')
      torch.save(bestModel, "/content/gdrive/MyDrive/Colab Notebooks/ACV_data/ENetB0"+mode+dataset+".pt")
      


    # Logs to TensorBoard
    log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

  # Compute final evaluation results
  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  # Logs to TensorBoard
  log_values(writer, epochs, train_loss, train_accuracy, "Train")
  log_values(writer, epochs, val_loss, val_accuracy, "Validation")
  log_values(writer, epochs, test_loss, test_accuracy, "Test")

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # Closes the logger
  writer.close()
  return net


In [ ]:
! rm -r runs

In [ ]:
B0_vanilla_SVHN = main(mode = 'vanilla', dataset = 'SVHN')
B0_advprop_SVHN = main(mode = 'advprop', dataset = 'SVHN')
B0_advprop2_SVHN = main(mode = 'advprop2', dataset = 'SVHN')

In [ ]:
B0_vanilla_CIFAR = main(mode = 'vanilla', dataset = 'CIFAR')
B0_advprop_CIFAR = main(mode = 'advprop', dataset = 'CIFAR')
B0_advprop2_CIFAR = main(mode = 'advprop2', dataset = 'CIFAR')

### Testing robustness

In [ ]:
def get_data_evaluation(test_batch_size=256): 
  # This function is needed to convert the PIL images to Tensors
  transform = list()
  transform.append(torchvision.transforms.ToTensor()) 
  #transform.append(torchvision.transforms.Lambda(lambda x: x.repeat(3,1,1)))
  transform.append(torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]))
  transform = torchvision.transforms.Compose(transform)

  # Load data
  #test_data = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True) 
  test_data = torchvision.datasets.SVHN('./data', split='test', transform=transform, download=True) 
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False) 
  return test_loader

In [ ]:
def get_mnist(test_batch_size=256): 
  # This function is needed to convert the PIL images to Tensors
  transform = list()
  transform.append(torchvision.transforms.ToTensor()) 
  transform.append(torchvision.transforms.Lambda(lambda x: x.repeat(3,1,1)))
  transform.append(torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]))
  transform = torchvision.transforms.Compose(transform)

  # Load data
  test_data = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True) 
  #test_data = torchvision.datasets.SVHN('./data', split='test', transform=transform, download=True) 
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False) 
  return test_loader

In [ ]:
def get_CIFAR(test_batch_size=256): 
  # This function is needed to convert the PIL images to Tensors
  transform = list()
  transform.append(torchvision.transforms.ToTensor()) 
  #transform.append(torchvision.transforms.Lambda(lambda x: x.repeat(3,1,1)))
  transform.append(torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]))
  transform = torchvision.transforms.Compose(transform)

  # Load data
  test_data = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True) 
  #test_data = torchvision.datasets.SVHN('./data', split='test', transform=transform, download=True) 
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False) 
  return test_loader

In [ ]:
# mnist_test_data = get_mnist() 
cifar_testdata = get_CIFAR()

In [ ]:
SVHN_valdata = get_data_evaluation()

In [ ]:
B0_vanilla_SVHN = torch.load("/content/gdrive/MyDrive/Colab Notebooks/ACV_data/ENetB0vanillaSVHN.pt")
B0_advprop_SVHN = torch.load("/content/gdrive/MyDrive/Colab Notebooks/ACV_data/ENetB0advpropSVHN.pt")
B0_advprop2_SVHN = torch.load("/content/gdrive/MyDrive/Colab Notebooks/ACV_data/ENetB0advprop2SVHN.pt")

B0_vanilla_CIFAR = torch.load("/content/gdrive/MyDrive/Colab Notebooks/ACV_data/ENetB0vanillaCIFAR.pt")  # normalization 0.5
B0_advprop_CIFAR = torch.load("/content/gdrive/MyDrive/Colab Notebooks/ACV_data/ENetB0advpropCIFAR.pt")  # normalization 0.5
B0_advprop2_CIFAR = torch.load("/content/gdrive/MyDrive/Colab Notebooks/ACV_data/ENetB0advpropCIFAR.pt")  # normalization 0.5

In [ ]:
bounds = (-1, 1)
foolbox_model_vanilla = fb.PyTorchModel(B0_vanilla_CIFAR.eval(), bounds = bounds)
foolbox_model_advprop = fb.PyTorchModel(B0_advprop_CIFAR.eval(), bounds = bounds)
foolbox_model_advprop2 = fb.PyTorchModel(B0_advprop2_CIFAR.eval(), bounds = bounds)

In [ ]:
def test_robustness(net,net_att, data_loader, device='cuda'):
  samples = 0.
  #cumulative_loss = 0.
  cumulative_accuracy = 0.
  # mean_accuracy=[]

  net.eval() 
  
  for batch_idx, (inputs, targets) in enumerate(data_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)
      # attack = fb.attacks.PGD(rel_stepsize = 1, steps = 1)
      # raw, clipped, is_adv = attack(net_att, inputs, targets, epsilons=0.0078)
      attack = fb.attacks.VirtualAdversarialAttack(steps=1, xi=0.01)
      raw, clipped, is_adv = attack(net_att, inputs, targets, epsilons=0.6)
      outputs = net(raw)

      samples+=inputs.shape[0]
      #cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_accuracy/samples*100

In [ ]:
print(test_robustness(B0_advprop_CIFAR, foolbox_model, cifar_testdata))

In [ ]:
def test_vanilla(net, data_loader, device='cuda'):
  samples = 0.
  #cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() 
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      outputs = net(inputs)
    

      # Apply the loss
      #loss = cost_function(outputs, targets)

      samples+=inputs.shape[0]
      #cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_accuracy/samples*100

In [ ]:
# test_SVHN(B0_vanilla_SVHN, SVHN_valdata)
test_vanilla(B0_advprop_CIFAR, cifar_testdata)

In [ ]:
test_vanilla(B0_vanilla_SVHN, mnist_test_data)